In [1]:
using Images
using Distributions

In [2]:
#出力層と隣接しているbackprop
function back_prop(l,δ,w,w_new,u,unit_sum,z,output_vector_size)
    for j in 1:unit_sum
        sum=0.0
        for  k  in 1:output_vector_size #outputvector
            sum=δ[l+1,k]*w[l+1,k,j]*dReLU(u[l,j])
        end
        δ[l,j]=sum
    end
    #incline=similar(w[l,:,:])
    incline=δ[l,:]*z[l-1,:]'
    #パラメータ更新
    η=0.1
    dw=-η*incline
    w_new[l,:,:]=w[l,:,:]+dw
end
#通常のbackprop
function back_prop(l,δ,w,w_new,u,unit_sum,z)
    for j in 1:unit_sum
        sum=0.0
        for  k  in 1:unit_sum
            sum=δ[l+1,k]*w[l+1,k,j]*dReLU(u[l,j])
        end
        δ[l,j]=sum
    end
    #incline=similar(w[l,:,:])
    incline=δ[l,:]*z[l-1,:]'
    #パラメータ更新
    η=0.1
    dw=-η*incline
    w_new[l,:,:]=w[l,:,:]+dw
end
#勾配の初期値
function firstbackprop(l,δ,w,w_new,u,z,output_vector_size,train)
    for j in 1:output_vector_size
        δ[l,1:output_vector_size]=z[l,1:output_vector_size]-train
    end
    incline=δ[l,1:output_vector_size]*z[l-1,:]'
     #パラメータ更新
    η=0.01
    dw=-η*incline
    w_new[l,1:output_vector_size,:]=w[l,1:output_vector_size,:]+dw
end

firstbackprop (generic function with 1 method)

In [3]:
#全結合層の手前のプーリング層の誤差
function back_prop_Nonparam(l,δ,w,w_new,u,unit_sum,z)
    for j in 1:unit_sum
        sum=0.0
        for  k  in 1:unit_sum
            sum=δ[l+1,k]*w[l+1,k,j]*1
        end
        δ[l,j]=sum
    end
    #incline=similar(w[l,:,:])
    #incline=δ[l,:]*z[l-1,:]'
    #パラメータ更新
    #η=0.1
    #dw=-η*incline
    #w_new[l,:,:]=w[l,:,:]+dw
end

#プーリング層の手前のユニットの誤差計算と重み更新
function first_conv_back_prop(l,δ,w,w_new,u,P,datay,datax,p_layer_num)
    i=1
    j=1
    for l in 1:datax
        for q in 1:P
            for k in 1:datay #このデータサイズはl+1のもの
                for p in 1:P
                    
                    δ_conv[l,i,j]+=δ_conv[l+1,k,l]*w[p_layer_num,k,l]*dReLU(u[l,i,j])
                    
                    i+=1
                end
            end
            j+=1
        end
    end
end


first_conv_back_prop (generic function with 1 method)

In [4]:
function ReLU(x)
    if(x>0)
        return x
    else
        return 0.0
    end
end

ReLU (generic function with 1 method)

In [5]:
m=Normal(0,1.0)
rand(m,4,4);

In [6]:
function dReLU(x)
    if(x>0)
        return 1.0
        else
        return 0.0
    end
end

dReLU (generic function with 1 method)

In [7]:
#function softmax(u::Array,output_vector_size)
#    z=similar(u)
#    u_sum=sum(exp.(u-maximum(u)))
#    for k in 1:output_vector_size
#        z[k]=exp(u[k]-maximum(u))/u_sum
#    end
#    return z
#end
function softmax(u::Array,output_vector_size)
    z=similar(u)
    u_sum=sum(exp.(u))
    for k in 1:output_vector_size
        z[k]=exp(u[k])/u_sum
    end
    return z
end

softmax (generic function with 1 method)

In [8]:
#畳み込み層
function convolution(l)
    x=Int(0)
    y=Int(0)
    for si in 0:S:datay-H
        y+=1
        x=0
        for sj in 0:S:datax-H
            x+=1
            for k in 1:K
                for i in 1:H
                    for j in 1:H
                        u[l,y,x]+=imgdata[k,i+si,j+sj]*h[i,j]+b[l,y,x]
                    end
                end
            end
            z[l,y,x]=ReLU(u[l,y,x])
        end
    end
    return z
end
#チャネルなし　2回め以降
function convolution(l,u::Array, z::Array)
    x=Int(0)
    y=Int(0)
    for si in 0:S:datay-H
        y+=1
        x=0
        for sj in 0:S:datax-H
            x+=1
                for i in 1:H
                    for j in 1:H
                        u[l,y,x]+=z[l-1,i+si,j+sj]*h[i,j]+b[l,y,x]
                    end
                end
            z[l,y,x]=ReLU(u[l,y,x])
        end
    end
    return z
end

function nuwral_net(l,z::Array,u,w)
    u[l,:]=w[l,:,:]*z[l-1,:]
    z[l,:]=ReLU.(u[l,:])
    return z
end

function nuwral_net(l,z::Array,u,w,output_vector_size)
    u[l,1:output_vector_size]=w[l,1:output_vector_size,:]*z[l-1,:]
    z[l,1:output_vector_size]=softmax(u[l,1:output_vector_size],output_vector_size)
    return z
end

nuwral_net (generic function with 2 methods)

In [9]:
function maxpooling(l,z,w,p_layer_num)
    x=Int(0)
    y=Int(0)
    for si in 0:S:datay-P
        y+=1
        x=0
        for sj in 0:S:datax-P
            x+=1
            z[l,y,x]=maximum(z[l-1,1+si:P+si,1+sj:P+sj])
            #一番大きい要素を持つ番号を求めて代入する
            for k in 1+si:P+si
                for t in 1+sj:P+sj
                    if(z[l-1,k,t]==z[l,y,x])
                        w[p_layer_num,k,t]=1.0
                    end
                end
            end
        end
    end
    return z
end

maxpooling (generic function with 1 method)

In [10]:
#チャネル数
K=3 #RGB
train=[0.0 for i in 1:40]
train[38]=1.0
data=load("phot/1berserk.jpg")
datax=3406 #画像サイズ
datay=3406
const Layernum=21
imgdata_raw=channelview(data) #[RGB,4608,3456]
imgdata=zeros(K,datay,datax);

In [11]:
#データの正規化
#分散を1に
#平均を0に
for k in 1:K
    datasum=sum(imgdata_raw[k,1:datay,1:datax])
    offset=datasum/(datax*datay)
    de=0.0
    for i in 1:datay
        for j  in 1:datax
            de+=(imgdata_raw[k,i,j]-offset)^2
        end
    end
    de=sqrt(de/(datay*datax))
    imgdata[k,:,:]=(imgdata_raw[k,1:datay,1:datax]-offset)/de
end

In [12]:
#first convolition layer
#factorizing convolution
H=3 #size H*H 
S=1 #stride
#valid padding
h=rand(m,H,H)*sqrt(2/(H*H)) #weight #He
b=rand(m,Layernum,Int((datay-H)/S+1),Int((datax-H)/S+1)) #bias
z=zeros(Float32,Layernum,Int((datay-H)/S+1),Int((datax-H)/S+1))
u=zeros(z)
total_p_layer=8
pooling_w=zeros(total_p_layer,datay,datax) #プーリング層の重み;

In [13]:
#畳み込み層1-1 第2層
l=2
@time z=convolution(l);

127.980981 seconds (5.00 G allocations: 83.689 GB, 4.89% gc time)


In [14]:
#パラメータ更新
datax=Int(1+(datax-H)/S)
datay=Int(1+(datay-H)/S)
H=3
h=rand(m,H,H)*sqrt(2/(H*H)) #weight #He
S=1
l=3
#畳み込み層1-2  第3層
@time z=convolution(l,u,z);

 45.147258 seconds (1.18 G allocations: 24.297 GB, 4.51% gc time)


In [15]:
#プーリング層 第4層
#パラメータ更新
datax=Int(1+(datax-H)/S)
datay=Int(1+(datay-H)/S)
S=3
P=3　#プーリングサイズ　P*P
p_layer_num=1
l=4
@time z=maxpooling(l,z,pooling_w,p_layer_num);

  3.371512 seconds (97.40 M allocations: 1.951 GB, 6.11% gc time)


In [16]:
#畳み込み層2-1 第5層
datax=Int(1+(datax-P)/S)
datay=Int(1+(datay-P)/S)
H=4
h=rand(m,H,H)*sqrt(2/(H*H)) #weight #He
S=1
l=5
@time z=convolution(l,u,z);

  7.834294 seconds (244.73 M allocations: 4.142 GB, 4.66% gc time)


In [17]:
#畳み込み層2-2 第6層
datax=Int(1+(datax-H)/S)
datay=Int(1+(datay-H)/S)
H=4
h=rand(m,H,H)*sqrt(2/(H*H)) #weight #He
S=1
l=6
@time z=convolution(l,u,z);

  7.826313 seconds (243.97 M allocations: 4.638 GB, 4.65% gc time)


In [18]:
#プーリング層 第7層
#パラメータ更新
datax=Int(1+(datax-H)/S)
datay=Int(1+(datay-H)/S)
S=2
P=2　#プーリングサイズ　P*P
l=7
p_layer_num=2
@time z=maxpooling(l,z,pooling_w,p_layer_num);

  0.522163 seconds (12.02 M allocations: 265.899 MB, 7.04% gc time)


In [19]:
#畳み込み層3-1 第8層
datax=Int(1+(datax-P)/S)
datay=Int(1+(datay-P)/S)
H=9
h=rand(m,H,H)*sqrt(2/(H*H)) #weight #He
S=1
l=8
@time z=convolution(l,u,z)
;

  8.187357 seconds (240.26 M allocations: 4.045 GB, 4.18% gc time)


In [20]:
#畳み込み層3-2 第9層
datax=Int(1+(datax-H)/S)
datay=Int(1+(datay-H)/S)
H=9
h=rand(m,H,H)*sqrt(2/(H*H)) #weight #He
S=1
l=9
@time z=convolution(l,u,z)
;

  7.956925 seconds (232.12 M allocations: 3.911 GB, 4.18% gc time)


In [21]:
#畳み込み層3-3 第10層
datax=Int(1+(datax-H)/S)
datay=Int(1+(datay-H)/S)
H=9
h=rand(m,H,H)*sqrt(2/(H*H)) #weight #He
S=1
l=10
@time z=convolution(l,u,z)
;

  7.726551 seconds (224.17 M allocations: 3.779 GB, 4.25% gc time)


In [22]:
#プーリング層 第11層
#パラメータ更新
datax=Int(1+(datax-H)/S)
datay=Int(1+(datay-H)/S)
S=2
P=2　#プーリングサイズ　P*P
l=11
p_layer_num=3
@time z=maxpooling(l,z,pooling_w,p_layer_num);

  0.115706 seconds (1.78 M allocations: 46.135 MB, 5.64% gc time)


In [23]:
#畳み込み層4-1 第12層
datax=Int(1+(datax-P)/S)
datay=Int(1+(datay-P)/S)
H=15
h=rand(m,H,H)*sqrt(2/(H*H)) #weight #He
S=1
l=12
@time z=convolution(l,u,z)
;

  4.620925 seconds (134.81 M allocations: 2.260 GB, 4.26% gc time)


In [24]:
#畳み込み層4-2 第13層
datax=Int(1+(datax-H)/S)
datay=Int(1+(datay-H)/S)
H=15
h=rand(m,H,H)*sqrt(2/(H*H)) #weight #He
S=1
l=13
@time z=convolution(l,u,z)
;

  4.130488 seconds (120.50 M allocations: 2.020 GB, 4.14% gc time)


In [25]:
#畳み込み層4-3第14層
datax=Int(1+(datax-H)/S)
datay=Int(1+(datay-H)/S)
H=15
h=rand(m,H,H)*sqrt(2/(H*H)) #weight #He
S=1
l=14
@time z=convolution(l,u,z)
;

  3.666752 seconds (106.95 M allocations: 1.793 GB, 4.18% gc time)


In [26]:
#プーリング層 第15層
#パラメータ更新
datax=Int(1+(datax-H)/S)
datay=Int(1+(datay-H)/S)
S=2
P=2　#プーリングサイズ　P*P
l=15
p_layer_num=4
@time z=maxpooling(l,z,pooling_w,p_layer_num);

  0.024770 seconds (299.14 k allocations: 7.939 MB, 12.18% gc time)


In [27]:
#畳み込み層5-1 第16層
datax=Int(1+(datax-P)/S)
datay=Int(1+(datay-P)/S)
H=33
h=rand(m,H,H)*sqrt(2/(H*H)) #weight #He
S=1
l=16
@time z=convolution(l,u,z)
;

  2.263468 seconds (66.76 M allocations: 1.105 GB, 4.11% gc time)


In [28]:
#畳み込み層5-2 第17層
datax=Int(1+(datax-H)/S)
datay=Int(1+(datay-H)/S)
H=33
h=rand(m,H,H)*sqrt(2/(H*H)) #weight #He
S=1
l=17
@time z=convolution(l,u,z)
;

  0.852992 seconds (24.67 M allocations: 420.633 MB, 4.56% gc time)


In [29]:
#畳み込み層5-3第18層
datax=Int(1+(datax-H)/S)
datay=Int(1+(datay-H)/S)
H=33
h=rand(m,H,H)*sqrt(2/(H*H)) #weight #He
S=1
l=18
@time z=convolution(l,u,z)
;

  0.113691 seconds (3.20 M allocations: 54.514 MB, 3.18% gc time)


In [30]:
#プーリング層 第19層
#パラメータ更新
datax=Int(1+(datax-H)/S)
datay=Int(1+(datay-H)/S)
S=2
P=2　#プーリングサイズ　P*P
l=19
p_layer_num=5
@time z=maxpooling(l,z,pooling_w,p_layer_num);

  0.000211 seconds (1.89 k allocations: 51.375 KB)


In [31]:
#出力ベクトルサイズ
FullyConnectLaynum=3
output_vector_size=40
output_vector=zeros(output_vector_size)

#パラメータ更新
datax=Int(1+(datax-P)/S)
datay=Int(1+(datay-P)/S)
u_fullyL=zeros(FullyConnectLaynum+1,datay*datax)
fl=1
z_fullyL=zeros(FullyConnectLaynum+1,datay*datax)
z_fullyL[fl,:]=reshape(z[l,1:datay,1:datax],datay*datax,1) #2次元のデータをベクトルに変更
δ=zeros(FullyConnectLaynum+1,datay*datax)
δ_conv=zeros(z)

#FullyConnectedLayer1　第20層
fl=2
#9に成
w_fullyL=rand(m,FullyConnectLaynum+1,datay*datax,datay*datax)*sqrt(2/(datax*datay)) #weight #He #fullyLayerWeight
w_fullyLnew=similar(w_fullyL)

@time z_fullyL=nuwral_net(fl,z_fullyL,u_fullyL,w_fullyL);

  0.195035 seconds (228.72 k allocations: 9.728 MB)


In [32]:
#FullyConnectedLayer2　第21層
#9に成
fl=3
@time z_fullyL=nuwral_net(fl,z_fullyL,u_fullyL,w_fullyL);

  0.000042 seconds (11 allocations: 54.438 KB)


In [33]:
#FullyConnectedLayer3　第22層  OUTPUTLAYER
#9に成
fl=4
@time z_fullyL=nuwral_net(fl,z_fullyL,u_fullyL,w_fullyL,output_vector_size);

  0.119741 seconds (107.26 k allocations: 4.572 MB)


In [35]:
#全結合の更新
@time firstbackprop(fl,δ,w_fullyL,w_fullyLnew,u_fullyL,z_fullyL, output_vector_size,train);

  0.490084 seconds (534.34 k allocations: 19.288 MB, 1.42% gc time)


In [36]:
#全結合の更新
fl=3
@time back_prop(fl,δ,w_fullyL,w_fullyLnew,u_fullyL,datax*datay,z_fullyL, output_vector_size);

  0.063785 seconds (12.43 k allocations: 723.577 KB)


In [37]:
#全結合の更新
fl=2
@time back_prop(fl,δ,w_fullyL,w_fullyLnew,u_fullyL,datax*datay,z_fullyL);

  0.031378 seconds (12.04 k allocations: 704.125 KB)


In [38]:
#全結合の更新
fl=1
@time back_prop_Nonparam(fl,δ,w_fullyL,w_fullyLnew,u_fullyL,datax*datay,z_fullyL,);

  0.016308 seconds (3.83 k allocations: 175.766 KB)


In [ ]:
#プーリング層の手前の層を更新
δ_conv[l,1:datay,1:datax]=reshape(δ[fl,:],datay,datax)

l=19
P=2
p_layer_num=5
first_conv_back_prop(l,δ,w,w_new,u,P,datay,datax,p_layer_num)

#datax=((datax-1)*S)+P #データサイズを戻していく
